In [1]:
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
# This is optional stuff - either pip install watermark
# or just comment it out (it just keeps track of what library
# versions I have)
%load_ext watermark
%watermark -iv

pyspark 2.4.1



In [3]:
# Comment these out to run on a cluster. Also, adjust memory to size of your laptop
pyspark.sql.SparkSession.builder.config('spark.driver.memory', '10g')
pyspark.sql.SparkSession.builder.config('spark.sql.shuffle.paritions', 5)

In [4]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# Checkpoint 1 

Read the csv file, drop the attributed_time (because I didn't use it in the MVP),
and downsample the 0 class by 50% because I'm still on my laptop 

In [50]:
if False:
    df = spark.read.csv('../data/train.csv', 
                    header=True, inferSchema=True)

    df = df.drop('attributed_time')
    df = df.sampleBy('is_attributed', fractions={0:.25,1:1.})
    
    test = spark.read.csv('../data/test.csv', 
                         header= True, inferSchema=True)

    df.write.parquet('../data/checkpoint1.parquet', mode='overwrite')
    test.write.parquet('../data/test_checkpoint1.parquet', mode='overwrite')
else:
    df = spark.read.parquet('../data/checkpoint1.parquet')
    test = spark.read.parquet('../data/test_checkpoint1.parquet')

In [51]:
df.dtypes

[('ip', 'int'),
 ('app', 'int'),
 ('device', 'int'),
 ('os', 'int'),
 ('channel', 'int'),
 ('click_time', 'timestamp'),
 ('is_attributed', 'int')]

In [52]:
test.dtypes

[('click_id', 'int'),
 ('ip', 'int'),
 ('app', 'int'),
 ('device', 'int'),
 ('os', 'int'),
 ('channel', 'int'),
 ('click_time', 'timestamp')]

# Checkpoint 2
## Counting 

Really taking stuff away from the MVP model here because it's too hard to 
make it run on my laptop.  Maybe next time...


In [53]:
from itertools import combinations
columns = [ 'device', 'os', 'ip', 'channel', 'app' ]

In [54]:
def make_count_column( sdf, sdf_name, groupby_clause ):
    column_name = "_".join(groupby_clause)
    table_name = "_".join([sdf_name, column_name])

    counts_sdf =  sdf.groupby( 
                        groupby_clause 
                ).count(
                ).withColumnRenamed(
                        'count', column_name
                ).orderBy(
                    column_name, ascending = False
                ).limit(
                    100
                )
    counts_sdf.createOrReplaceTempView(table_name)
    
    join_clauses = [ f"{sdf_name}.{b} = {table_name}.{b}" for b in groupby_clause ]
    return counts_sdf, join_clauses

if True:
    test_frames = {}
    train_frames = {}
    
    for bigram in combinations(columns, 2):
        b = list(bigram)
        c = '_'.join(b)
        
        train_frames[bigram] = make_count_column( df, "train", b )
        test_frames[bigram]  = make_count_column( test, "test", b )
        

In [55]:
df.createOrReplaceTempView('train')
test.createOrReplaceTempView('test')

In [56]:
count_names = [ "_".join(c) for c in combinations(columns, 2)]
bigram_select = ", ".join(count_names)
    
bigram_select

'device_os, device_ip, device_channel, device_app, os_ip, os_channel, os_app, ip_channel, ip_app, channel_app'

In [61]:
train_select = (f"""
SELECT is_attributed, {bigram_select}
FROM train
""")

where = "WHERE " 
for b in count_names:
    where += b + " IS NOT NULL OR\n"
where = where[:-3]
    
join_train = ""
for k in train_frames.keys():
    b = "train_" +  "_".join(k)
    join_train += "JOIN " + b  + " ON \n"
    for col in train_frames[k][1]:
        join_train +=  col + " AND\n "
    join_train = join_train[:-5]
          
train_select += join_train + where
print(train_select)


SELECT is_attributed, device_os, device_ip, device_channel, device_app, os_ip, os_channel, os_app, ip_channel, ip_app, channel_app
FROM train
JOIN train_device_os ON 
train.device = train_device_os.device AND
 train.os = train_device_os.os JOIN train_device_ip ON 
train.device = train_device_ip.device AND
 train.ip = train_device_ip.ip JOIN train_device_channel ON 
train.device = train_device_channel.device AND
 train.channel = train_device_channel.channel JOIN train_device_app ON 
train.device = train_device_app.device AND
 train.app = train_device_app.app JOIN train_os_ip ON 
train.os = train_os_ip.os AND
 train.ip = train_os_ip.ip JOIN train_os_channel ON 
train.os = train_os_channel.os AND
 train.channel = train_os_channel.channel JOIN train_os_app ON 
train.os = train_os_app.os AND
 train.app = train_os_app.app JOIN train_ip_channel ON 
train.ip = train_ip_channel.ip AND
 train.channel = train_ip_channel.channel JOIN train_ip_app ON 
train.ip = train_ip_app.ip AND
 train.app = tr

In [62]:
df = spark.sql(train_select)


In [63]:
df.show(100)

+-------------+---------+---------+--------------+----------+-----+----------+-------+----------+------+-----------+
|is_attributed|device_os|device_ip|device_channel|device_app|os_ip|os_channel| os_app|ip_channel|ip_app|channel_app|
+-------------+---------+---------+--------------+----------+-----+----------+-------+----------+------+-----------+
|            0|  1126278|   253102|       1765322|   5159353| 9940|     83384| 168220|     13483| 28056|     915840|
|            0| 10854303|   253102|       2106576|   3836133|62610|    529118| 990791|     14957| 25558|    1445321|
|            0|  9615116|    42946|        637666|   5159353| 9674|    134495|1134463|      8748| 11231|     660173|
|            0|  1126278|   267793|       1765322|   5159353|10635|     83384| 168220|     14428| 30359|     915840|
|            0| 10854303|   253102|       2106576|   3836133|62610|    529118| 990791|     14957| 25558|    1445321|
|            0|  1126278|   267793|       1765322|   5159353|106